**Firstly, Setup the runtime as GPU**







In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [3]:
from google.colab import files
uploaded = files.upload()

Saving pride and prejudice.txt to pride and prejudice.txt


In [5]:
file = open("pride and prejudice.txt", "r", encoding = "utf8")
lines = []
for i in file:
    lines.append(i)
data = ""
for i in lines:
  data = ' '. join(lines)
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space
data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of Pride and Prejudice This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title:'

In [6]:
len(data)

733690

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer, open('token.pkl', 'wb'))
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
len(sequence_data)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)
y[:15]

7254
The Length of sequences are:  131175
Data:  [[   1  181  390]
 [ 181  390 1000]
 [ 390 1000    3]
 [1000    3  298]
 [   3  298    4]
 [ 298    4  946]
 [   4  946   41]
 [ 946   41 1000]
 [  41 1000   23]
 [1000   23   21]]
Response:  [1000    3  298    4  946   41 1000   23   21    1]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             72540     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7254)              7261254   
                                                                 
Total params: 20382794 (77.75 MB)
Trainable params: 20382794 (77.75 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=20, batch_size=64, callbacks=[checkpoint])

Epoch 1/20
2050/2050 [==============================] - ETA: 0s - loss: 1.5003
Epoch 1: loss improved from inf to 1.50028, saving model to next_words.h5
2050/2050 [==============================] - 46s 21ms/step - loss: 1.5003
Epoch 2/20
2050/2050 [==============================] - ETA: 0s - loss: 1.1916
Epoch 2: loss improved from 1.50028 to 1.19161, saving model to next_words.h5
2050/2050 [==============================] - 35s 17ms/step - loss: 1.1916
Epoch 3/20
2050/2050 [==============================] - ETA: 0s - loss: 1.0643
Epoch 3: loss improved from 1.19161 to 1.06425, saving model to next_words.h5
2050/2050 [==============================] - 36s 17ms/step - loss: 1.0643
Epoch 4/20
2050/2050 [==============================] - ETA: 0s - loss: 0.9687
Epoch 4: loss improved from 1.06425 to 0.96868, saving model to next_words.h5
2050/2050 [==============================] - 36s 18ms/step - loss: 0.9687
Epoch 5/20
2050/2050 [==============================] - ETA: 0s - loss: 0.8971
E

In [12]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break

  print(predicted_word)
  return predicted_word
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)

          Predict_Next_Words(model, tokenizer, text)

      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: Title: Pride and Prejudice
['Pride', 'and', 'Prejudice']
1/1 [==============================] - 1s 868ms/step
apology
Enter your line: Language: English
['Language:', 'English']
1/1 [==============================] - 1s 615ms/step
distance
Enter your line: START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE 
['AND', 'PREJUDICE', '']
1/1 [==============================] - 0s 19ms/step
in
Enter your line: 0
Execution completed.....
